<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/Crypto_Binance_Historical_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [1]:
# Install openpyxl and CCXT
!pip install openpyxl ccxt

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 1.3MB 18.4MB/s 
     |████████████████████████████████| 2.6MB 41.3MB/s 
     |████████████████████████████████| 256kB 40.7MB/s 
     |████████████████████████████████| 143kB 33.2MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 235kB 53.6MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=9ce8cede5631d50727ad57d792b9641cb0d8a5bdb832148f99b34267df12bd6d
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26310 sha256=ddec27541ba7c17ebc35c1deac082f63369c4a8273ab66a1d0e51fd06de90663
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built idna-ssl typing


In [2]:
import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)


In [ ]:
scrape_candles_to_csv('btc_usdt_1m.csv', 'binance', 3, 'BTC/USDT', '1m', '2019-01-0100:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)

1000 BTC/USDT candles in total from 2021-01-03T19:59:00.000Z to 2021-01-04T12:38:00.000Z
2000 BTC/USDT candles in total from 2021-01-03T03:19:00.000Z to 2021-01-04T12:38:00.000Z
3000 BTC/USDT candles in total from 2021-01-02T10:39:00.000Z to 2021-01-04T12:38:00.000Z
4000 BTC/USDT candles in total from 2021-01-01T17:59:00.000Z to 2021-01-04T12:38:00.000Z
5000 BTC/USDT candles in total from 2021-01-01T01:19:00.000Z to 2021-01-04T12:38:00.000Z
6000 BTC/USDT candles in total from 2020-12-31T08:39:00.000Z to 2021-01-04T12:38:00.000Z
7000 BTC/USDT candles in total from 2020-12-30T15:59:00.000Z to 2021-01-04T12:38:00.000Z
8000 BTC/USDT candles in total from 2020-12-29T23:19:00.000Z to 2021-01-04T12:38:00.000Z
9000 BTC/USDT candles in total from 2020-12-29T06:39:00.000Z to 2021-01-04T12:38:00.000Z
10000 BTC/USDT candles in total from 2020-12-28T13:59:00.000Z to 2021-01-04T12:38:00.000Z
11000 BTC/USDT candles in total from 2020-12-27T21:19:00.000Z to 2021-01-04T12:38:00.000Z
12000 BTC/USDT cand